In [1]:
cd /code/interpretable-han-for-document-classification-with-keras

/code/interpretable-han-for-document-classification-with-keras


In [2]:
!pip install --upgrade pip

     |████████████████████████████████| 2.1MB 26.4MB/s eta 0:00:01
  Found existing installation: pip 19.1.1
    Uninstalling pip-19.1.1:
      Successfully uninstalled pip-19.1.1


In [3]:
!pip install numpy==1.18

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 72.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
    Uninstalling numpy-1.19.2:
      Successfully uninstalled numpy-1.19.2


In [4]:
!python /code/interpretable-han-for-document-classification-with-keras/setup.py install

running install
running bdist_egg
running egg_info
writing hierachical_attention_network_for_document_classification.egg-info/PKG-INFO
writing dependency_links to hierachical_attention_network_for_document_classification.egg-info/dependency_links.txt
writing requirements to hierachical_attention_network_for_document_classification.egg-info/requires.txt
writing top-level names to hierachical_attention_network_for_document_classification.egg-info/top_level.txt
reading manifest file 'hierachical_attention_network_for_document_classification.egg-info/SOURCES.txt'
writing manifest file 'hierachical_attention_network_for_document_classification.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
copying han/AoS3.py -> build/lib/han
creating build/bdist.linux-x86_64
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/han
copying build/lib/han/attention.py -> build/bdist.linux-x86_64/egg/han
copying build/li

In [5]:
"""
This code is for 3 level Hierarchichal Attention model
"""


import re
import numpy as np
import pandas as pd
import logging
import sys
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from nltk.tokenize import sent_tokenize
from sklearn.model_selection import train_test_split

!pip install keras-self-attention

from han.AoS3 import HAN


from keras_self_attention import SeqSelfAttention
import nltk
nltk.download('punkt')

# Create a logger to provide info on the state of the
# script
stdout = logging.StreamHandler(sys.stdout)
stdout.setFormatter(logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
))
logger = logging.getLogger('default')
logger.setLevel(logging.INFO)
logger.addHandler(stdout)


max_doc_num = 5
max_sent_length = 100
max_sent_num = 25
MAX_VOC_SIZE = 20000
GLOVE_DIM = 200
TEST_SPLIT = 0.1

Special_value=0


#####################################################
# Pre processing                                    #
#####################################################
logger.info("Pre-processsing data.")


data = pd.read_csv('/data/Sample_data.csv', encoding='latin1')


df=data.iloc[:,1:max_doc_num+1]
reviews = data.iloc[:,1:max_doc_num+1].values
target = data['sentiment'].values

df['concat'] = pd.Series(df.fillna('').values.tolist()).map(lambda x: ''.join(map(str,x)))
#del data


#####################################################
# Tokenization                                      #
#####################################################
logger.info("Tokenization.")

# Build a Keras Tokenizer that can encode every token
word_tokenizer = Tokenizer(num_words=MAX_VOC_SIZE)
word_tokenizer.fit_on_texts(df['concat'])




X = np.zeros((len(data), max_doc_num , max_sent_num, max_sent_length), dtype='int32')

#Tokenizing each patient interview to create embedding matrix

for i, review in enumerate(reviews):
  for j in range(len(review)):
      if not pd.isnull(reviews[i,j]):
        #word_tokenizer.fit_on_texts(review[j])
        sentences = sent_tokenize(reviews[i,j])
        tokenized_sentences = word_tokenizer.texts_to_sequences(
          sentences
        )
        tokenized_sentences = pad_sequences(
            tokenized_sentences, maxlen=max_sent_length
        )

        pad_size = max_sent_num - tokenized_sentences.shape[0]

        if pad_size < 0:
            tokenized_sentences = tokenized_sentences[0:max_sent_num]
        else:
            tokenized_sentences = np.pad(
                tokenized_sentences, ((0,pad_size),(0,0)),
                mode='constant', constant_values= Special_value
            )
      
        X[i,j] = tokenized_sentences[None, ...]
      else:
        #X[i,j].fill(np.nan)
        X[i,j]=0
      

# Transform the labels into a format Keras can handle
y = to_categorical(target)



#####################################################
# Word Embeddings                                   #
#####################################################
logger.info(
    "Creating embedding matrix using pre-trained GloVe vectors."
)

# Now, we need to build the embedding matrix. For this we use
# a pretrained (on the wikipedia corpus) 100-dimensional GloVe
# model.

# Load the embeddings from a file
embeddings = {}
with open('/data/glove.6B.%dd.txt' % GLOVE_DIM, encoding='utf-8') as file:
    for line in file:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')

        embeddings[word] = coefs

# Initialize a matrix to hold the word embeddings
embedding_matrix = np.random.random(
    (len(word_tokenizer.word_index) + 1, GLOVE_DIM)
)

# Let the padded indices map to zero-vectors. This will
# prevent the padding from influencing the results
embedding_matrix[0] = 0

# Loop though all the words in the word_index and where possible
# replace the random initalization with the GloVe vector.
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector



  Preparing metadata (setup.py) ... done
  Created wheel for keras-self-attention: filename=keras_self_attention-0.51.0-py3-none-any.whl size=18912 sha256=fe4c4d983cc07eb873458c71d2aca5fa8624168d32266b32fb7dc4f9717e44ea
  Stored in directory: /root/.cache/pip/wheels/95/b1/a8/5ee00cc137940b2f6fa198212e8f45d813d0e0d9c3a04035a3
Successfully built keras-self-attention


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


2023-09-18 14:36:20,262 - default - INFO - Pre-processsing data.
2023-09-18 14:36:20,270 - default - INFO - Tokenization.
2023-09-18 14:36:20,442 - default - INFO - Creating embedding matrix using pre-trained GloVe vectors.


In [6]:
h_model = HAN(embedding_matrix, max_sent_length=100, max_sent_num=25, max_doc_num=5, word_embed_dim=100, sent_embed_dim=100, doc_embed_dim=100)


2023-09-18 14:36:43.821067: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-18 14:36:43.844116: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2499995000 Hz
2023-09-18 14:36:43.844493: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55f125b60df0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-09-18 14:36:43.844516: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [7]:
checkpoint_path='/code/model.{epoch:02d}-{val_loss:.2f}.hdf5'

In [8]:
#####K-fold Cross Validation#####
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
TEST_SPLIT=0.1

X_train, X_test, y_train, y_test= train_test_split(X, target, test_size=TEST_SPLIT, random_state=0, stratify=y)


y_test=to_categorical(y_test)

kFold = StratifiedKFold(n_splits=10)
scores = []
idx = 0
for train, val in kFold.split(X_train, y_train):
  idx = idx+1
  print("fold:", idx)
  X_tr=X_train[train]
  y_tr=to_categorical(y_train[train])
  X_val=X_train[val]
  y_val=to_categorical(y_train[val])
  
  R=h_model.train_model(checkpoint_path, np.array(X_tr), np.array(y_tr), np.array(X_val), np.array(y_val), np.array(X_test), np.array(y_test),optimizer=tf.keras.optimizers.Adagrad(
    learning_rate=0.001), epochs=20,batch_size=20)
  scores.append(R)
  del  X_tr, X_val, y_tr, y_val
  #del h_model

print(scores)
scores_df=pd.DataFrame(scores)
scores_df.to_csv('/results/scores_AoS3.csv')  
#print(scores.mean())

fold: 1
Epoch 1/20


2023-09-18 14:37:22.353111: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 3200000000 exceeds 10% of free system memory.
2023-09-18 14:37:22.370231: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 3200000000 exceeds 10% of free system memory.
2023-09-18 14:37:25.704247: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 3200000000 exceeds 10% of free system memory.


1/2 [==============>...............] - ETA: 0s - loss: 0.6934 - acc: 0.6500

2023-09-18 14:37:28.728911: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 2880000000 exceeds 10% of free system memory.
2023-09-18 14:37:28.729293: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 2880000000 exceeds 10% of free system memory.


2/2 [==============================] - ETA: 0s - loss: 0.6975 - acc: 0.5000
Epoch 00001: val_loss improved from inf to 0.69464, saving model to /code/model.01-0.69.hdf5
2/2 [==============================] - 9s 4s/step - loss: 0.6975 - acc: 0.5000 - val_loss: 0.6946 - val_acc: 0.6000
Epoch 2/20
2/2 [==============================] - ETA: 0s - loss: 0.6965 - acc: 0.5000
Epoch 00002: val_loss did not improve from 0.69464
2/2 [==============================] - 6s 3s/step - loss: 0.6965 - acc: 0.5000 - val_loss: 0.6948 - val_acc: 0.6000
Epoch 3/20
2/2 [==============================] - ETA: 0s - loss: 0.6965 - acc: 0.5000
Epoch 00003: val_loss did not improve from 0.69464
2/2 [==============================] - 6s 3s/step - loss: 0.6965 - acc: 0.5000 - val_loss: 0.6948 - val_acc: 0.6000
Epoch 4/20
2/2 [==============================] - ETA: 0s - loss: 0.6965 - acc: 0.5000
Epoch 00004: val_loss did not improve from 0.69464
2/2 [==============================] - 5s 3s/step - loss: 0.6965 - ac

In [9]:
accuracy=[]
for i in range (9):
  accuracy.append(scores[i][1])
  

In [10]:
mean_accuracy=np.mean(accuracy)
print(mean_accuracy)

0.4000000059604645
